In [2]:
import numpy as np
from scipy.optimize import minimize

In [93]:
n = 10
a = np.array([np.random.uniform(3, 7) for _ in range(n)])

t_i = lambda f_i, c_i, a_i: a_i + (f_i/c_i)**3
T = lambda f, c, a=a: np.sum(t_i(fi, ci, ai)*fi for fi, ci, ai in zip(f, c, a))

int_t_i = lambda f_i, c_i, a_i: a_i * f_i + (f_i)**3/(4 * c_i**3)
z = lambda f, c, a=a: np.sum(int_t_i(f_i, c_i, a_i) for f_i, c_i, a_i in zip(f, c, a))

In [47]:
def get_k_best(l, c, k=5):
    """
    Finds k best rows in l
    
    takes:
        l -- matrix of T_i, f, c
        k -- k best rows
    returns:
        k best rows
    """
    mat = l
    sor = c[np.argsort(mat)]
    return sor[:k]

In [68]:
def cross_breed(vects, constrain, k=5):
    """
    Computes cross breeing of vectors
    
    takes:
        vects      -- vectors, that new generation would be created from
        constrain  -- (float) size of new vector space (kinda)
        k          -- amount vectors that would be used to produce new child
        
    returns:
        new vector
    """
    coeff = np.random.dirichlet(np.ones(vects.shape[0]), size=1).reshape(-1)
    
    conv_mat = np.array([ci * vi for ci, vi in zip(coeff, vects)])
    new_vec = conv_mat.sum(axis=0)
    return new_vec

In [69]:
def gen_population(dim_pop, dim_c=n, C=150, costs=None, Ts = None):
    """
    Generates population. 
    On first step, when no c initialized, generates random c. 
    On next steps calculates c using cross of gens.
    
    takes: 
        dim_pop -- number of populations (how many rows in returned matrix)
        dim_c   -- dimention of population (how many columns in returned matrix)
        C       -- max cost sum (sum(c_i) <= C)
        costs   -- passed if it is desired to calculate not on first step
    returns:
        matrix c[dim_pop, dim_c]
    """
    c = np.zeros((dim_pop, dim_c))
    if costs is None:
        for i in range(dim_pop):
            c[i] = np.array([np.random.uniform(0, C/dim_c) for j in range(dim_c)])
    else:
        for i in range(dim_pop):
            # number of used vects
            k = 5
            k_best = get_k_best(Ts, costs, k)
            c[i] = cross_breed(k_best, 2, k)
    return c

In [94]:
def mineze(c, F = 500, m=n):
    """
    Calculates step of inner minimization. 
    
    takes:
        c -- vector of costs
        F -- total flow of network
        m -- number of connections in graph
        
    returns:
        tuple: T_i, f, c
        T_i -- value in min point f
        f   -- found min with specified c
        c   -- passed c
    """
    cons = ({'type' : 'eq',  'fun' : lambda x: np.sum(x) - F}, 
            {'type' : 'ineq','fun' : lambda x: x            })
    f = np.ones(n) * F/m

    f = minimize(lambda x: z(x, c), f, constraints=cons).x
    T_i = T(f, c)
    return T_i

In [101]:
from sklearn.metrics import euclidean_distances

c = None
Tfc = None
T_prev = 0
prec = 1

epoch = 0
while prec > 1e-3:
    c = gen_population(10, costs=c, Ts=Tfc)
    Tfc = []
    for c_i in c:
        Tfc.append(mineze(c_i))

    prec = euclidean_distances(sorted(Tfc)[0], T_prev)
    T_prev = sorted(Tfc)[0]
    epoch = epoch + 1
    
    ind = np.argsort(Tfc)[0]
    print('epoch: {}'.format(epoch))
    print('T value: {}'.format(Tfc[ind]))
    print('costs: {}'.format(c[ind]))
    print('precision: {}\n'.format(prec))

epoch: 1
T value: 134794.43159784097
costs: [  3.87521321   9.59211054   5.56710582   7.33967691  14.76360158
   4.64170755  13.15569147   8.72882046  11.7854591    4.53350119]
precision: [[ 134794.43159784]]

epoch: 2
T value: 130489.5082321689
costs: [  5.93082808   8.39563489   4.32439595   7.08002547  13.64612834
   6.53236318  12.86328453   9.55667545  10.55869392   6.54471289]
precision: [[ 4304.92336567]]

epoch: 3
T value: 130691.22463360384
costs: [  6.4853557    7.97607079   4.06674511   7.49390374  13.04692435
   7.62079854  12.62183731  10.16950873   9.64048146   7.67317884]
precision: [[ 201.71640143]]

epoch: 4
T value: 131697.51610029276
costs: [  6.74447326   7.74587095   3.96454478   7.81111409  12.69656411
   8.03719047  12.44068206  10.30155726   9.2744669    8.20687941]
precision: [[ 1006.29146669]]

epoch: 5
T value: 131883.6937132713
costs: [  6.87988836   7.73214141   3.95310172   7.83281656  12.57417508
   8.04825944  12.3815322   10.30909793   9.23714254   8.27